<a href="https://colab.research.google.com/github/eyaler/avatars4all/blob/master/melaflefon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wav2Lip Demo Colab

### Made just a little bit more accessible by Eyal Gruss (https://eyalgruss.com, eyalgruss@gmail.com)

##### Original project: http://bhaasha.iiit.ac.in/lipsync

##### Original notebook: https://colab.research.google.com/drive/1tZpDWXz49W6wDcTprANRGLo2D_EbD5J8

##### Combined with First Order Motion Model: https://colab.research.google.com/github/eyaler/avatars4all/blob/master/fomm_bibi.ipynb

In [ ]:
#@title Dub it! { run: "auto" }
#@markdown 1. Choose audio (you can also enter a YouTube or similar URL):
audio = 'Dangerous time' #@param ['Dangerous time', 'Go home', 'מלפפונים חמוצים', 'שונאת שמאלנים', 'אני שולה', 'Grab from uploaded video'] {allow-input: true}
#@markdown 2. Press the play button on the left
#@markdown 3. Press "Browse" below, and upload image(s) or video(s)

from google.colab import files
%cd /content/sample_data
images = files.upload()

if images:
  %tensorflow_version 1.x
  %cd /content
  !git clone --depth 1 https://github.com/eyaler/Wav2Lip.git
  import os
  if not os.path.exists('/content/Wav2Lip/checkpoints/wav2lip_gan.pth'):
    !gdown https://drive.google.com/uc?id=1dwHujX7RVNCvdR1RR93z0FS2T2yzqup9 -O /content/Wav2Lip/checkpoints/wav2lip_gan.pth
  !wget --no-check-certificate -nc https://eyalgruss.com/fomm/wav2lip_gan.pth -O /content/Wav2Lip/checkpoints/wav2lip_gan.pth
  !wget --no-check-certificate -nc https://eyalgruss.com/fomm/melaflefon.mp3
  !wget --no-check-certificate -nc https://eyalgruss.com/fomm/sonet.mp3
  !wget --no-check-certificate -nc https://eyalgruss.com/fomm/shoula.mp3
  !wget --no-check-certificate -nc https://eyalgruss.com/fomm/gohome.mp3
  !wget --no-check-certificate -nc https://eyalgruss.com/fomm/s3fd-619a316812.pth -O /content/Wav2Lip/face_detection/detection/sfd/s3fd.pth
  !pip install -U youtube-dl
  !youtube-dl --extract-audio --audio-format mp3 https://www.youtube.com/watch?v=cQ54GDm1eL0 -o /content/obama.mp3
  grab = False
  if '://' in audio:
    if os.path.exists('/content/custom.mp3'):
      os.remove('/content/custom.mp3')
    !youtube-dl --extract-audio --audio-format mp3 '$audio' -o /content/custom.mp3
    audio = 'custom'
  elif audio=='Dangerous time':
    audio = 'obama'
  elif audio=='Go home':
    audio = 'gohome'
  elif audio == 'מלפפונים חמוצים':
    audio = 'melaflefon'
  elif audio == 'שונאת שמאלנים':
    audio = 'sonet'
  elif audio == 'אני שולה':
    audio = 'shoula'
  elif audio == 'Grab from uploaded video':
    grab = True
  audio = '/content/'+audio+'.mp3'

  %cd /content/Wav2Lip
  outputs = []
  for im in images:
    infile = '/content/sample_data/'+im
    ext = infile.rsplit('.',1)[1]
    if ext != ext.lower() or "'" in infile:
      lower = infile.rsplit('.',1)[0].replace("'",'')+'.'+ext.lower()
      os.rename(infile, lower)
      infile = lower
      ext = ext.lower()
    outfile = '/content/'+im.rsplit('.')[0].replace("'",'')+'_out.mp4'
    if grab:
      audio = infile
    !python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face '$infile' --audio '$audio' --pads 0 20 0 0 --outfile \"'$outfile'\" 2>&1 | tee /content/out.txt
    bad = !cat /content/out.txt | grep 'Face not detected'
    if bad:
      import cv2
      print('\nFace not detected - will use whole frame')
      if ext in ['jpg', 'png', 'jpeg']:
        frame = cv2.imread(infile)
      else:
        video_stream = cv2.VideoCapture(infile)
        still_reading, frame = video_stream.read()
      x1 = 0
      h,x2 = frame.shape[:2]
      if x2>h:
        x1 = (x2-h)//2
        x2 = x1+h
      !python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face '$infile' --audio '$audio' --box 0 $h $x1 $x2 --pads 0 20 0 0 --outfile \"'$outfile'\"
    outputs.append(outfile)
    os.remove(infile)

  from IPython.display import HTML, clear_output
  from base64 import b64encode

  clear_output()
  for file in outputs:
    with open(file, 'rb') as f:
      data_url = "data:video/mp4;base64," + b64encode(f.read()).decode()
    display(HTML("""
    <video width=600 controls autoplay loop>
          <source src="%s" type="video/mp4">
    </video>""" % data_url))

  for file in outputs:
    try:
      files.download(file)
    except:
      pass